# NER - BioBERT: Disease Identification in a text

## Set Up

In [2]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

## Data Download

In [3]:
from datasets import load_dataset
raw_datasets = load_dataset('EMBO/BLURB', 'NCBI-disease-IOB')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for EMBO/BLURB contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/EMBO/BLURB.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

Before the download


Generating validation split: 0 examples [00:00, ? examples/s]

Before the download


Generating test split: 0 examples [00:00, ? examples/s]

Before the download


In [ ]:
#train, validation and test datasets
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 5425
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 924
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 941
    })
})

In [13]:
#first sentence and the corresponding NER tags

print(raw_datasets["train"][0]["tokens"])
print(raw_datasets["train"][0]["ner_tags"])
print(len(raw_datasets["train"][0]["ner_tags"]))

['Identification', 'of', 'APC2', ',', 'a', 'homologue', 'of', 'the', 'adenomatous', 'polyposis', 'coli', 'tumour', 'suppressor', '.']
[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0]
14


In [14]:
#tag id and tag names
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-Disease', 'I-Disease'], id=None), length=-1, id=None)

In [15]:
#store tag names to label names
label_names = ner_feature.feature.names
label_names

['O', 'B-Disease', 'I-Disease']



*   O indicates the token doesn’t correspond to disease entity.
*   B- indicates the beginning of an entity.
*   I- indicates a token is contained inside the same entity (e.g., the “York” token is a part of the “New York” entity).







In [16]:
#first sentence and the corresponding NER tags (in a better way)
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Identification of APC2 , a homologue of the adenomatous polyposis coli      tumour    suppressor . 
O              O  O    O O O         O  O   B-Disease   I-Disease I-Disease I-Disease O          O 


In [35]:
#downlond biobert from hugging space
from transformers import AutoTokenizer

model_checkpoint = "dmis-lab/biobert-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Data Preprocessing

### Tokenization

In [36]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())
print(len(inputs.tokens()))

['[CLS]', 'I', '##dent', '##ification', 'of', 'AP', '##C', '##2', ',', 'a', 'ho', '##mo', '##logue', 'of', 'the', 'ad', '##eno', '##mat', '##ous', 'p', '##oly', '##po', '##sis', 'co', '##li', 't', '##umour', 'suppress', '##or', '.', '[SEP]']
31


The tokenizer added the special tokens used by the model ([CLS] at the beginning and [SEP] at the end) and breaks most of the words. This introduces a mismatch between our inputs and the labels: the list of labels has only 14 elements, whereas our input now has 31 tokens. Accounting for the special tokens is easy (as they are at the beginning and the end), but we also need to make sure we align all the labels with the proper words.

In [37]:
print(inputs.word_ids())

[None, 0, 0, 0, 1, 2, 2, 2, 3, 4, 5, 5, 5, 6, 7, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 11, 11, 12, 12, 13, None]


We can then expand our label list to match the tokens.


*   The first rule we’ll apply is that special tokens get a label of -100. This is because by default -100 is an index that is ignored in the loss function we will use (cross entropy)
*   Then, each token gets the same label as the token that started the word it’s inside, since they are part of the same entity.
*   For tokens inside a word but not at the beginning, we replace the B- with I- (since the token does not begin the entity)




### Aligning labels with tokens

In [38]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [39]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, -100]


The function added the -100 for the two special tokens at the beginning and the end, and a new 0 for our word that was split into two tokens

In [40]:
#function for data preprcoessing for all instances
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [41]:
#preprcocessing the whole data into using the map function
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/5425 [00:00<?, ? examples/s]

Map:   0%|          | 0/924 [00:00<?, ? examples/s]

Map:   0%|          | 0/941 [00:00<?, ? examples/s]

In [42]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5425
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 924
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 941
    })
})

### Padding

In [43]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [44]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
print(batch["labels"])

tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    1,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    0,    0,    0, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100],
        [-100,    0,    1,    2,    2,    2,    2,    2,    2,    2,    2,    2,
            2,    2,    2,    2,    2,    2,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0, -100]])


In [45]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, -100]
[-100, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


The first set of labels has been padded to the length of the second one using -100s.

## Evaluation Metric

In [46]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=d631a41b465f54011e6198c0969ef31cafef3c181578fb53df1ef3cff0f2617a
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [47]:
import evaluate

metric = evaluate.load("seqeval")

For more details on seqeval:

https://github.com/chakki-works/seqeval


In [48]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [49]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [50]:
print(id2label)
print(label2id)

{0: 'O', 1: 'B-Disease', 2: 'I-Disease'}
{'O': 0, 'B-Disease': 1, 'I-Disease': 2}


In [51]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dmis-lab/biobert-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
model.config.num_labels

3

## Trainer API

We have used early stopping and weight decay to prevent overfitting

In [ ]:
from transformers import TrainingArguments,Trainer,EarlyStoppingCallback

training_args = TrainingArguments(
    "biobert-finetuned-ner",
    num_train_epochs=10,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    warmup_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model = 'f1',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 4)]
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.054097,0.723454,0.787802,0.754258,0.981981
2,0.256400,0.048373,0.810036,0.861499,0.834975,0.985684
3,0.030300,0.052868,0.807229,0.851334,0.828695,0.984813
4,0.030300,0.062105,0.823815,0.861499,0.842236,0.985311
5,0.008900,0.061667,0.818074,0.874206,0.845209,0.986275
6,0.004600,0.075531,0.823944,0.891995,0.856620,0.985871
7,0.004600,0.081681,0.818824,0.884371,0.850336,0.986338
8,0.001500,0.083667,0.827751,0.879288,0.852742,0.986867


## Test Set Evaluation

In [1]:
logits, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(logits, axis=-1)

# Remove ignored index (special tokens) and covert to labels
true_predictions = [
    [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

NameError: name 'trainer' is not defined

## Inference

In [ ]:
import pandas as pd

def tag_sentence(text:str):
    # convert our text to a  tokenized sequence
    inputs = tokenizer(text, truncation=True, return_tensors="pt").to("cuda")
    # get outputs
    outputs = model(**inputs)
    # convert to probabilities with softmax
    probs = outputs[0][0].softmax(1)
    # get the tags with the highest probability
    word_tags = [(tokenizer.decode(inputs['input_ids'][0][i].item()), id2label[tagid.item()])
                  for i, tagid in enumerate (probs.argmax(axis=1))]

    return pd.DataFrame(word_tags, columns=['word', 'tag'])

In [ ]:
text = """Clustering of missense mutations in the ataxia - telangiectasia gene in a sporadic T - cell leukaemia ."""

print(tag_sentence(text))

         word        tag
0       [CLS]          O
1           C          O
2    ##luster          O
3       ##ing          O
4          of          O
5        miss          O
6      ##ense          O
7   mutations          O
8          in          O
9         the          O
10         at  B-Disease
11       ##ax  I-Disease
12       ##ia  I-Disease
13          -  I-Disease
14         te  I-Disease
15     ##lang  I-Disease
16       ##ie  I-Disease
17       ##ct  I-Disease
18      ##asi  I-Disease
19        ##a  I-Disease
20       gene          O
21         in          O
22          a          O
23          s  B-Disease
24  ##poradic  I-Disease
25          T  I-Disease
26          -  I-Disease
27       cell  I-Disease
28         le  I-Disease
29      ##uka  I-Disease
30     ##emia  I-Disease
31          .          O
32      [SEP]          O


In [ ]:
from transformers.pipelines.token_classification import AggregationStrategy
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "./biobert-finetuned-ner/checkpoint-1360"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint,aggregation_strategy="simple"
)

In [ ]:
token_classifier("Clustering of missense mutations in the ataxia - telangiectasia gene in a sporadic T - cell leukaemia.")

[{'entity_group': 'Disease',
  'score': 0.9995025,
  'word': 'ataxia - telangiectasia',
  'start': 40,
  'end': 63},
 {'entity_group': 'Disease',
  'score': 0.99272454,
  'word': 'sporadic T - cell leukaemia',
  'start': 74,
  'end': 101}]